In [ ]:
from datasets import load_dataset_builder, get_dataset_split_names, get_dataset_config_names, load_dataset
from transformers import TrainingArguments, XLMRobertaForSequenceClassification, Trainer
from util import *
import json, pprint
import pandas as pd

pp = pprint.PrettyPrinter(indent = 4)

In [ ]:
model_name = 'xlm-roberta-base'
model = XLMRobertaForSequenceClassification.from_pretrained(model_name, num_labels=3)

dataset_name = 'assin'
configs = get_dataset_config_names(dataset_name)
splits = get_dataset_splits_and_configs(dataset_name, configs)

pp.pprint(f'{dataset_name}\'s configs: {configs}')
pp.pprint(f'{dataset_name}\'s splits: {splits}')

# 1. Training in different Configurations
Above , we see Assin is splitted in *train, test and validation* sets for  every configuration. We will perform the training on every configuration:
- pt-br;
- pt-pt;
- full.

Therefore, we'll train three models and compare their results.

## 1.1 pt-br

In [ ]:
config = 'ptbr'

pt_br_train = load_dataset(dataset_name, config, split = 'train')
pt_br_train = pt_br_train.rename_column('entailment_judgment', 'label')
tokenized_train = pt_br_train.map(xlmR_tokenizer, batched=True)

pt_br_test = load_dataset(dataset_name, config, split = 'test')
pt_br_test = pt_br_test.rename_column('entailment_judgment', 'label')
tokenized_test = pt_br_test.map(xlmR_tokenizer, batched=True)

In [ ]:
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics,
)

trainer.train()
